<a href="https://colab.research.google.com/github/tnssn/NotionTools/blob/main/%E5%85%A8%E5%87%BA%E3%81%97_Drive%E7%89%88_notionDB%2B%E3%82%B5%E3%83%96%E3%83%9A%E3%83%BC%E3%82%B8%E6%9C%AC%E6%96%87%E3%81%AEcsv%E3%82%92%E4%BD%9C%E3%82%8B%E3%82%84%E3%81%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello, Notion API!")

Hello, Notion API!


In [ ]:
import os
import requests
import json
import pandas as pd
from datetime import datetime
from google.colab import drive

# 🔹 Google Drive をマウント
drive.mount('/content/drive')

# ここにAPIキーをセット（Colabのセッションごとに入力が必要）
NOTION_API_KEY = os.environ["NOTION_API_KEY"]

# 🔹 データベースIDと対応するファイル名を設定
DATABASES = {
    "1a9c1a389044806f9825e2e713d02e62": "シーンログ",
    "1acc1a38904480d0acf1fabe9728f5fc": "設定DB"
}

# APIヘッダー
headers = {
    "Authorization": f"Bearer {os.environ['NOTION_API_KEY']}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json"
}

# 🔹 Google Drive の保存先フォルダ
drive_folder = "/content/drive/MyDrive/NotionLogs"
os.makedirs(drive_folder, exist_ok=True)

# 🔹 全データベースのデータを統合するリスト
all_records = []

# 🔥 各データベースをループ処理
for database_id, custom_name in DATABASES.items():
    print(f"🔄 データベース {database_id}（{custom_name}）のデータを取得中…")

    # データベースのページを取得
    url = f"https://api.notion.com/v1/databases/{database_id}/query"
    response = requests.post(url, headers=headers)
    data = response.json()

    # 結果を格納するリスト
    records = []

    # 各ページの情報を取得
    for page in data["results"]:
        page_id = page["id"]
        properties = page["properties"]  # すべてのプロパティを取得

        # **全プロパティを辞書に格納**
        record = {"データベース名": custom_name}  # どのデータベースかを記録

        for prop_name, prop_value in properties.items():
            if prop_value["type"] == "title":
                record[prop_name] = prop_value["title"][0]["text"]["content"] if prop_value["title"] else ""
            elif prop_value["type"] == "rich_text":
                record[prop_name] = prop_value["rich_text"][0]["text"]["content"] if prop_value["rich_text"] else ""
            elif prop_value["type"] == "select":
                record[prop_name] = prop_value["select"]["name"] if prop_value["select"] else ""
            elif prop_value["type"] == "multi_select":
                record[prop_name] = ", ".join([option["name"] for option in prop_value["multi_select"]])
            elif prop_value["type"] == "date":
                record[prop_name] = prop_value["date"]["start"] if prop_value["date"] else ""
            elif prop_value["type"] == "checkbox":
                record[prop_name] = str(prop_value["checkbox"])
            elif prop_value["type"] == "number":
                record[prop_name] = str(prop_value["number"])
            elif prop_value["type"] == "url":
                record[prop_name] = prop_value["url"] if prop_value["url"] else ""
            elif prop_value["type"] == "email":
                record[prop_name] = prop_value["email"] if prop_value["email"] else ""
            elif prop_value["type"] == "phone_number":
                record[prop_name] = prop_value["phone_number"] if prop_value["phone_number"] else ""
            else:
                record[prop_name] = "[未対応のプロパティ]"

        # ページのブロック（本文）を取得
        block_url = f"https://api.notion.com/v1/blocks/{page_id}/children"
        block_response = requests.get(block_url, headers=headers)
        block_data = block_response.json()

        # 本文を取得して結合
        content_list = []
        if "results" in block_data:
            for block in block_data["results"]:
                if block["type"] == "paragraph":
                    rich_text = block["paragraph"].get("rich_text", [])
                    if rich_text:
                        content_list.append(rich_text[0]["text"]["content"])

        record["本文"] = "\n".join(content_list) if content_list else "本文なし"

        # データをリストに追加
        records.append(record)
        all_records.append(record)  # 🔥 すべてのデータベースのデータを統合

    # **個別のCSVに保存**
    df = pd.DataFrame(records)
    today_str = datetime.today().strftime("%Y%m%d")
    csv_filename = f"{today_str}_{custom_name}.csv"  # 🔹 カスタム名をファイル名に！
    drive_csv_path = f"{drive_folder}/{csv_filename}"
    df.to_csv(drive_csv_path, index=False)
    print(f"✅ {csv_filename} を Google Drive に保存しました！")

# **統合データを1つのCSVに保存**
df_all = pd.DataFrame(all_records)
all_csv_path = f"{drive_folder}/{today_str}_全データベース.csv"
df_all.to_csv(all_csv_path, index=False)
print(f"✅ すべてのデータを統合して {all_csv_path} に保存しました！")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔄 データベース 1a9c1a389044806f9825e2e713d02e62（シーンログ）のデータを取得中…
✅ 20250307_シーンログ.csv を Google Drive に保存しました！
🔄 データベース 1acc1a38904480d0acf1fabe9728f5fc（設定DB）のデータを取得中…
✅ 20250307_設定DB.csv を Google Drive に保存しました！
✅ すべてのデータを統合して /content/drive/MyDrive/NotionLogs/20250307_全データベース.csv に保存しました！
